In [1]:
import numpy as np
import time
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import gzip

df = pd.read_table("SNV/MANIFEST.txt")
fileslist = []
header = []
for file_name in df[0:33]["filename"]:
    fileslist.append(file_name)
with gzip.open("SNV/"+fileslist[0],"rb") as f:
    print(f)
    i = 0
    while i <=4:
        header.append(f.readline())
        i += 1
    mf = pd.read_csv(f, delimiter = "\t")
    table = mf[0:]

<gzip _io.BufferedReader name='SNV/02747363-f04a-4ba6-a079-fe4f87853788/TCGA.UCS.mutect.02747363-f04a-4ba6-a079-fe4f87853788.DR-10.0.somatic.maf.gz' 0x1f485db5bb0>


In [2]:
#Remove columns with all row values NaN
new_table = table.dropna(axis=1,how="all") 
print(new_table.isna().any().sum())
new_table.info(verbose=True)

54
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 102 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    Hugo_Symbol                  object 
 1    Entrez_Gene_Id               int64  
 2    Center                       object 
 3    NCBI_Build                   object 
 4    Chromosome                   object 
 5    Start_Position               int64  
 6    End_Position                 int64  
 7    Strand                       object 
 8    Variant_Classification       object 
 9    Variant_Type                 object 
 10   Reference_Allele             object 
 11   Tumor_Seq_Allele1            object 
 12   Tumor_Seq_Allele2            object 
 13   dbSNP_RS                     object 
 14   dbSNP_Val_Status             object 
 15   Tumor_Sample_Barcode         object 
 16   Matched_Norm_Sample_Barcode  object 
 17   Tumor_Validation_Allele1     object 
 18   Tumor_Validation_Alle

In [3]:
#Handle features and genes_id#

#Retrieve Y (genes_id) and X (all other features)
Y = new_table.loc[:,["Hugo_Symbol"]].values
length = len(Y)

#Retrive features
X = new_table.drop(["Hugo_Symbol"], axis = 1)
thresh = length*0.90
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 101 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    Entrez_Gene_Id               int64  
 1    Center                       object 
 2    NCBI_Build                   object 
 3    Chromosome                   object 
 4    Start_Position               int64  
 5    End_Position                 int64  
 6    Strand                       object 
 7    Variant_Classification       object 
 8    Variant_Type                 object 
 9    Reference_Allele             object 
 10   Tumor_Seq_Allele1            object 
 11   Tumor_Seq_Allele2            object 
 12   dbSNP_RS                     object 
 13   dbSNP_Val_Status             object 
 14   Tumor_Sample_Barcode         object 
 15   Matched_Norm_Sample_Barcode  object 
 16   Tumor_Validation_Allele1     object 
 17   Tumor_Validation_Allele2     object 
 18   Mutation_Status         

In [4]:
#Handling NaN in cloumns below treshold 0.90(90%)#
X  = X.dropna(axis=1,thresh=int(thresh))
#X.isna().sum().sort_values(ascending=False)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Data columns (total 58 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Entrez_Gene_Id               10449 non-null  int64  
 1   Center                       10449 non-null  object 
 2   NCBI_Build                   10449 non-null  object 
 3   Chromosome                   10449 non-null  object 
 4   Start_Position               10449 non-null  int64  
 5   End_Position                 10449 non-null  int64  
 6   Strand                       10449 non-null  object 
 7   Variant_Classification       10449 non-null  object 
 8   Variant_Type                 10449 non-null  object 
 9   Reference_Allele             10449 non-null  object 
 10  Tumor_Seq_Allele1            10449 non-null  object 
 11  Tumor_Seq_Allele2            10449 non-null  object 
 12  dbSNP_RS                     9507 non-null   object 
 13  Tumor_Sample_Bar

In [14]:
#Fill NaN values with interpolation method pad-by using existing values#
X = X.interpolate(method="pad")
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Columns: 119508 entries, Entrez_Gene_Id to GDC_Validation_Status_Valid
dtypes: bool(1), float64(1), int64(10), uint8(119496)
memory usage: 1.2 GB


In [13]:
obj_df = X.select_dtypes(include=['object'])
X=pd.get_dummies(data=X,columns=obj_df.columns)
X.info()
X

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10449 entries, 0 to 10448
Columns: 119508 entries, Entrez_Gene_Id to GDC_Validation_Status_Valid
dtypes: bool(1), float64(1), int64(10), uint8(119496)
memory usage: 1.2 GB


,Entrez_Gene_Id,Start_Position,End_Position,t_depth,t_ref_count,t_alt_count,n_depth,ALLELE_NUM,TRANSCRIPT_STRAND,TSL,...,case_id_d9952e44-b573-4854-893e-b3bbc5287f2f,case_id_ddfdd05e-058c-4877-b389-724a6c101353,case_id_ebb834c6-374e-4077-a4ce-c74a850be358,case_id_ee6083aa-9453-44ff-8cc5-79b71fea289d,case_id_eec002c4-7602-4179-8eb3-2b0508d5ee1c,case_id_f2cc142b-9fc0-44d9-9ae3-558b4eace375,case_id_f98c183e-aaa1-40c4-83e7-c248cc3adb55,case_id_ffedc8be-1056-4205-b9d9-99b5bdb872db,GDC_Validation_Status_Unknown,GDC_Validation_Status_Valid
0,6536,44008472,44008472,103,65,38,132,1,-1,1.0,...,0,0,0,0,0,0,0,0,0,1
1,1263,44803280,44803280,84,51,33,112,1,1,1.0,...,0,0,0,0,0,0,0,0,0,1
2,199920,56740711,56740711,22,15,7,23,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
3,2629,155233116,155233116,46,29,17,96,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
4,83417,157581530,157581530,60,33,27,79,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10444,6247,18642139,18642139,102,48,54,119,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
10445,7317,47201524,47201524,43,32,11,47,1,1,1.0,...,0,0,0,0,0,0,0,0,1,0
10446,441531,77969434,77969434,205,112,93,216,1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
10447,9737,102657224,102657224,199,97,102,186,1,1,2.0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
Y = LabelEncoder().fit_transform(Y.ravel())
Y

array([5426, 4497,  734, ..., 4374, 2387, 4526])

In [17]:
X = StandardScaler().fit_transform(X)
X

array([[-0.08414803, -0.61689314, -0.61689315, ..., -0.07599565,
        -1.72511663,  1.72511663],
       [-0.08483798, -0.60296662, -0.60296662, ..., -0.07599565,
        -1.72511663,  1.72511663],
       [-0.0588445 , -0.39380041, -0.39380041, ..., -0.07599565,
         0.57967095, -0.57967095],
       ...,
       [-0.02723065, -0.02183333, -0.02183333, ..., -0.07599565,
         0.57967095, -0.57967095],
       [-0.08372919,  0.4107431 ,  0.4107431 , ..., -0.07599565,
         0.57967095, -0.57967095],
       [-0.06997897,  1.29269292,  1.29269292, ..., -0.07599565,
         0.57967095, -0.57967095]])